In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from google.cloud import bigquery
%load_ext google.cloud.bigquery

dataset = 'tmp_sandbox'
client = bigquery.Client(project='exponea-ai-development', location='EU')
 
import plotly.graph_objs as go
from plotly.tools import set_credentials_file, set_config_file
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

init_notebook_mode(connected=True)
set_credentials_file(username='bahylkubo', api_key='RSaJFeCLv06HN3MIk6Uq')
set_config_file(world_readable=False, sharing='private')

# Ambulance cars
Loading data, analyses and graphs

In [5]:
%%bigquery doprava
SELECT *
FROM `tmp_sandbox.doprava`

In [7]:
doprava.columns = [
    'row',
    'provider_id',
    'lab_id',
    'patient_id',
    'job_id',
    'disease_id',
    'date',
    'date2',
    'dist',
    'price_med',
    'price',
    'patient_state',
    'has_friend',
    'trip_id',
    'car_id',
    'patients',
    'is_urgent',
    'doctor',
    'doctor_info',
    'lab',
    'lab_info',
    'was_inspected',
    'money_back'
]
doprava.head()

,row,provider_id,lab_id,patient_id,job_id,disease_id,date,date2,dist,price_med,...,trip_id,car_id,patients,is_urgent,doctor,doctor_info,lab,lab_info,was_inspected,money_back
0,2017015214016,100000121743,100000226048,100000021527,LSPP_D,F40.9,12.09.2017,None,32000,160000,...,293,1612,0,A,NaN,NaN,NaN,NaN,0,0
1,201701521402,100000121743,100000226048,100000123265,LSPP_D,H82,03.09.2017,None,32000,160000,...,229,436,0,A,NaN,NaN,NaN,NaN,0,0
2,2017015214014,100000121743,100000226048,100000207777,LSPP_D,M53.0,10.09.2017,None,32000,160000,...,285,436,0,A,NaN,NaN,NaN,NaN,0,0
3,201701521403,100000121743,100000226048,100000123278,LSPP_D,F43.0,03.09.2017,None,32000,160000,...,230,436,0,A,NaN,NaN,NaN,NaN,0,0
4,2017015214019,100000121743,100000226048,100000331894,LSPP_D,J45.8,15.09.2017,None,34000,170000,...,313,436,0,A,NaN,NaN,NaN,NaN,0,0


In [8]:
# transformations
doprava.dist = doprava.dist / 1000
doprava.price = doprava.price / 10000
doprava.price_med = doprava.price_med / 10000
doprava.date = pd.to_datetime(doprava.date, format='%d.%m.%Y')
doprava = doprava.sort_values(by='date')

In [9]:
# Sub dataset
doprava_sub = doprava[[
    'date',
    'provider_id', 
    'car_id', 
    'trip_id',
    'patient_id', 
    'patients', 
    'is_urgent', 
    'has_friend', 
    'disease_id', 
    'dist', 
    'price'
]]
doprava_sub.head()

,date,provider_id,car_id,trip_id,patient_id,patients,is_urgent,has_friend,disease_id,dist,price
49386,2017-01-01,100000015785,1357,317,100000093489,1,A,N,Y04.9,1.0,0.79
44712,2017-01-01,100000006866,330,40,100000548528,1,A,N,R10.4,18.0,14.22
33907,2017-01-01,100000006940,116,271,100000250096,1,N,N,R51,15.0,12.00
22730,2017-01-01,100000008025,1502,3,100000833917,1,A,A,J04.0,15.0,11.85
45823,2017-01-01,100000015785,921,623,100000562786,1,A,N,R10.4,5.0,3.95


## Basic analyses

In [10]:
len(doprava)

551094

In [11]:
print('Automatic inspection', doprava.was_inspected.sum())
print('Not inspected', len(doprava) - doprava.was_inspected.sum())
print('Refunded money', round(doprava.price.sum(), 2))
print('Saved money', round(doprava.money_back.sum(), 2))

Automatic inspection 1192
Not inspected 549902
Refunded money 8091892.7
Saved money 0


### Providers

In [12]:
print('Companies: ', len(doprava_sub.provider_id.unique()))
cars = doprava_sub.groupby('provider_id')['car_id'].nunique()
cars = pd.DataFrame(cars).sort_values(by='car_id', ascending=False)
print('Mean cars: ', cars.car_id.mean())

patients = doprava_sub.groupby('provider_id')['patients'].sum()
patients = pd.DataFrame(patients).sort_values(by='patients', ascending=False)
print('Mean patients: ', patients.patients.mean())

Companies:  107
Mean cars:  15.794392523364486
Mean patients:  9695.084112149532


### Cars

In [14]:
print('Cars: ', len(doprava.car_id.unique()))
people = doprava_sub.groupby('car_id')['patients'].sum()
people = pd.DataFrame(people).sort_values(by='patients', ascending=False)
print('Mean num of patients: ', people.patients.mean())
dists = doprava_sub.groupby('car_id')['dist'].sum()
dists = pd.DataFrame(dists).sort_values(by='dist', ascending=False)
print('Mean dist: ', dists.dist.mean())

Cars:  1620
Mean num of patients:  640.3543209876543
Mean dist:  7437.835185185185


### Extremes

In [16]:
doprava_sub[['patients', 'dist', 'price']].describe()

,patients,dist,price
count,551094.000000,551094.000000,551094.000000
mean,1.882390,21.864315,14.683326
std,1.804306,36.049094,80.775800
min,0.000000,0.000000,0.000000
25%,1.000000,5.000000,3.160000
50%,1.000000,14.000000,8.690000
75%,2.000000,25.000000,15.800000
max,24.000000,1705.000000,13162.500000


In [20]:
doprava_sub.price.quantile(0.999)

283.92

In [24]:
# Extreme price
doprava_sub[doprava_sub.price > doprava_sub.price.quantile(0.999)].sort_values(by='price', ascending=False).head()

,date,provider_id,car_id,trip_id,patient_id,patients,is_urgent,has_friend,disease_id,dist,price
9395,2017-05-25,100000012429,1385,10502,100000849674,1,A,N,D10.9,351.0,13162.5
9507,2018-07-05,100000012429,1387,4208,100000852940,1,A,N,Q44.2,349.0,13087.5
9506,2017-09-15,100000012429,1388,4501,100000852940,1,A,N,Q44.2,349.0,13087.5
9622,2018-01-18,100000012429,1387,208,100000852940,1,A,N,Q44.2,276.0,10350.0
9651,2018-09-13,100000012429,1388,6208,100000812406,1,A,N,Z94.4,269.0,10087.5


In [26]:
# Extreme distance
doprava_sub[doprava_sub.dist > doprava_sub.dist.quantile(0.999)].sort_values(by='dist', ascending=False).head()

,date,provider_id,car_id,trip_id,patient_id,patients,is_urgent,has_friend,disease_id,dist,price
59064,2017-05-18,100000006773,604,2400,100000244471,3,N,A,C80.0,1705.0,864.44
22860,2018-08-09,100000006787,903,7661,100000338871,3,N,N,D33.0,1694.0,1101.10
22859,2018-07-22,100000006787,903,7073,100000338871,1,N,N,D33.2,1694.0,1101.10
4374,2018-04-16,100000015785,1138,467410,100000411263,1,N,N,M41.80,1415.0,707.50
17728,2017-08-28,100000006787,903,9280,100000800747,1,N,N,G81.9,1396.0,707.77


## Revenues / Patients / Cars

In [41]:
# Revenues
provider_revenues = pd.DataFrame(doprava_sub.groupby('provider_id')['price'].sum()).reset_index()
provider_cars = pd.DataFrame(doprava_sub.groupby('provider_id')['car_id'].nunique()).reset_index()
provider_patients = pd.DataFrame(doprava_sub.groupby('provider_id')['patients'].sum()).reset_index()


providers = provider_revenues \
    .merge(provider_cars, on='provider_id') \
    .merge(provider_patients, on='provider_id') \
    .sort_values(by='price', ascending=False) \
    .reset_index(drop=True)

providers.columns = ['provider_id', 'price_total', 'cars', 'patients_total']

iplot([go.Bar(x=providers.index, y=providers.price_total, name='Revenue'),
       go.Bar(x=providers.index, y=providers.patients_total, name='Patients'),
       go.Bar(x=providers.index, y=providers.cars, name='Cars')])
# provider_revenue.index

In [42]:
x = providers.patients_total
y = providers.price_total
size = providers.cars

fig = go.Figure()
fig.add_scatter(x=x,
                y=y,
                mode='markers',
                marker={'size': size,
                        'opacity': 0.6,
                        'colorscale': 'Viridis'
                       })
iplot(fig)

In [44]:
print('Unique providers', len(doprava_sub.provider_id.unique()))
print('Unique patients', len(doprava_sub.patient_id.unique()))
print('Unique cars', len(doprava_sub.patient_id.unique()))

Unique providers 107
Unique patients 70324
Unique cars 70324


In [57]:
# picked car
car = doprava_sub[doprava_sub['car_id'] == 236][['car_id', 'date', 'patients']].sort_values(by='date')
iplot([go.Bar(x=car.date, y=car.patients)])

# Use-case: Overloaded ambulances

In [64]:
patients_counts = pd.DataFrame(doprava_sub.groupby('patient_id')['trip_id'].nunique()).reset_index()
patients_diseases = pd.DataFrame(doprava_sub.groupby('patient_id')['disease_id'].nunique()).reset_index()

patients = patients_counts.merge(patients_diseases, on='patient_id')

patients.columns = ['patient_id', 'trips', 'diseases']
patients = patients.sort_values('diseases', ascending=False)
max_count = 365
patients_flag = patients[patients.trips > max_count]
patients_flag.head()

,patient_id,trips,diseases
45205,100000547753,585,49
14702,100000158341,691,31
3776,100000041325,651,30
21045,100000225119,414,29
14657,100000157892,563,24


# Use-case: Overloaded medics 

In [69]:
doprava_sub['doctor'] = doprava['doctor']
has_doctor = doprava_sub[doprava_sub.doctor.notnull()]
doctor_counts = pd.DataFrame(has_doctor.groupby('doctor')['trip_id'].count())
doctor_disease = pd.DataFrame(has_doctor.groupby('doctor')['disease_id'].apply(set))
doctor_providers = pd.DataFrame(has_doctor.groupby('doctor')['provider_id'].apply(set))
doctor_price = pd.DataFrame(has_doctor.groupby('doctor')['price'].sum())

doctors = doctor_counts \
    .merge(doctor_disease, on='doctor') \
    .merge(doctor_providers, on='doctor') \
    .merge(doctor_price, on='doctor') \
    .reset_index()

doctors.columns = ['doctor', 'trips', 'diseases', 'providers', 'price']
doctors['disease_count'] = doctors.diseases.apply(lambda row: len(row))

doctors = doctors.sort_values(by='trips',ascending=False).reset_index(drop=True)
doctors.head(10)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,doctor,trips,diseases,providers,price,disease_count
0,1.000000e+11,5554,"{C00.0, F20.9, N18.5, N18.1}","{100000115522, 100000024605}",43218.54,4
1,1.000001e+11,5220,"{Z01.4, K04.5, I73.9, J18.9, M81.00, N18.5}",{100000006808},27642.89,6
2,1.000001e+11,5061,"{G62.88, Z49.0, Z01.9, N20.0, E32.0, R50.2, X4...","{100000010744, 100000006943}",65315.46,61
3,1.000000e+11,3585,"{N18.2, M54.10, L97, F70.1, N18.9, M18.9, N18.5}",{100000115522},29985.07,7
4,1.000001e+11,3135,"{N18.9, I10.00, N18.5}",{100000115522},22079.85,3
5,1.000001e+11,3017,"{Z94.0, N11.8, I10.90, I25.9, N08.3, N18.1, N1...",{100000006787},24546.72,8
6,1.000000e+11,2987,"{R07.4, N18.5, N18.2}",{100000115522},22727.23,3
7,1.000001e+11,2599,"{Z89.7, E11.91, N08.3, N18.1, N18.5}",{100000015785},20300.36,5
8,1.000000e+11,2575,"{I85.0, S73.10, C85.1, C16.0, C19, F41.9, F10....","{100000023865, 100000006787, 100000104916}",36020.96,288
9,1.000001e+11,2466,"{N18.5, S72.10, G52.9}",{100000022665},17031.28,3


In [70]:
%%bigquery choroby
SELECT
string_field_0 AS disease_id,
string_field_1 AS disease_name
FROM `tmp_sandbox.cis_dgn`

In [71]:
print('N18.5:', choroby[choroby.disease_id == 'N18.5'].disease_name.values[0])
print('I25.0:', choroby[choroby.disease_id == 'I25.0'].disease_name.values[0])

N18.5: Chronická choroba obliciek, 5. štádium
I25.0: Aterosklerotická srdcovocievna choroba, tak oznacená


# Use-case: Overloaded providers

In [72]:
provider_counts = pd.DataFrame(doprava_sub.groupby('provider_id')['trip_id'].count())
provider_doctor = pd.DataFrame(doprava_sub.groupby('provider_id')['doctor'].nunique())
provider_disease = pd.DataFrame(doprava_sub.groupby('provider_id')['disease_id'].nunique())
provider_price = pd.DataFrame(doprava_sub.groupby('provider_id')['price'].sum())
provider_dist = pd.DataFrame(doprava_sub.groupby('provider_id')['dist'].sum())

providers = provider_counts \
    .merge(provider_doctor, on='provider_id') \
    .merge(provider_disease, on='provider_id') \
    .merge(provider_price, on='provider_id') \
    .merge(provider_dist, on='provider_id') \
    .reset_index()

providers.columns = ['provider_id', 'trips', 'doctors', 'diseases', 'price', 'dist']
providers = providers.sort_values(by='price', ascending=False).reset_index(drop=True)
providers.head(10)

,provider_id,trips,doctors,diseases,price,dist
0,100000015785,118883,711,3066,1.561928e+06,2207750.0
1,100000115522,60108,1136,2512,9.462317e+05,1842660.0
2,100000006943,95405,518,2203,9.215097e+05,1166468.0
3,100000012429,313,0,170,8.280504e+05,21906.0
4,100000006783,57476,0,2021,5.127685e+05,649074.0
5,100000006951,29061,258,1389,3.582481e+05,493051.0
6,100000006787,12822,589,1121,2.150763e+05,414876.0
7,100000006790,12902,0,1011,1.703003e+05,215570.0
8,100000005028,9436,123,860,1.476244e+05,273860.0
9,100000106661,7353,103,657,1.393422e+05,269015.0


# ML approach

In [79]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import ExtraTreesClassifier

/usr/local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning:

numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.



In [78]:
doprava_sub['was_inspected'] = doprava.was_inspected
train = doprava_sub[['provider_id', 'patients', 'is_urgent', 'has_friend', 'dist', 'price']]
train.is_urgent = train.is_urgent.map({'A': 1, 'N': 0})
train.has_friend = train.has_friend.map({'A': 1, 'N': 0})

test = doprava_sub['was_inspected']

X_train, X_test, Y_train, Y_test = train_test_split(train, test, test_size=0.5, random_state=123)

tree = DecisionTreeClassifier(criterion = "entropy", 
                              random_state = 100, 
                              max_depth=5, 
                              min_samples_leaf=3,
                              min_samples_split=10)
tree.fit(X_train, Y_train)

Y_pred = tree.predict(X_test)
Y_prob = tree.predict_proba(X_test)[:,1]

tree.report = classification_report(Y_test, Y_pred, target_names=["no","yes"])

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [81]:
extra = ExtraTreesClassifier(criterion = "entropy", 
                             random_state = 100,
                             n_estimators = 100,
                             max_depth=12, 
                             min_samples_leaf=3,
                             min_samples_split=10,
                             bootstrap=False)
extra.fit(X_train, Y_train)

# display the relative importance of each attribute
iplot([go.Bar(x=train.columns, y=extra.feature_importances_)])